In [1]:
using LightGraphs
#Graphs.jl is absolutely terrible; you can't even remove edges!
#Arun Jambulapati, HW #3, CME 257 submission: LightGraphs

In [2]:
#brief exhibit of the features of this package

test = Graph(10)
#create a 10 node graph
test1 = Graph(10, 32)
#create another 10 node graph with 32 randomly chosen edges

add_edge!(test, 4, 5)
#add an edge between 4 and 5

for i in edges(test1)
    println(i)
end
#print the edges of test
rem_edge!(test, 4, 5)
#remove an edge from test

print("Neighbors of 4: ")
println(neighbors(test1, 4))
#print the neighborhood of vertex 4

#compute the shortest paths from vertex 4 to all other nodes
print("Distances from 4: ")
println(dijkstra_shortest_paths(test1, 4).dists)

#set test to test1's complement
test = complement(test1)

#mincut of test1
print("Mincut: ")
println(mincut(test1))



edge 4 - 3
edge 1 - 4
edge 9 - 5
edge 5 - 4
edge 10 - 5
edge 3 - 6
edge 8 - 3
edge 3 - 5
edge 8 - 6
edge 6 - 4
edge 4 - 9
edge 8 - 5
edge 7 - 1
edge 8 - 9
edge 2 - 7
edge 9 - 7
edge 10 - 3
edge 6 - 7
edge 6 - 9
edge 8 - 1
edge 3 - 9
edge 1 - 10
edge 5 - 6
edge 2 - 8
edge 7 - 8
edge 10 - 8
edge 6 - 10
edge 1 - 9
edge 1 - 3
edge 4 - 8
edge 4 - 10
edge 4 - 2
Neighbors of 4: [8,2,6,1,9,5,3,10]
Distances from 4: [1,1,1,0,1,1,2,1,1,1]
Mincut: (Bool[true,false,true,true,true,true,true,true,true,true],3)


In [3]:
n = 250000
p_1 = .00001
p_2 = .002
G = erdos_renyi(n, p_1, is_directed=false)
H = erdos_renyi(round(Int,3*sqrt(n)), p_2, is_directed=false)
G = union(G,H)
#generate a random graph with a large set of relatively sparse edges and a smaller dense cluster of edges
threshold = round(Int, .5*(length(edges(G)))^.5)
#triangle detection low-degree high-degree threshold
#changing the probabilities and graphs if you want to test different densities and sizes
#generating this graph takes a while due to the number of random bits required; a better test graph would require
#much more work
#the reason this takes so long is because to generate the several billion random bits required to create the graph, 
#we have to do several cpu cycles to get each of them
#if this takes too long, just change the graph size to something smaller

281

In [4]:
neighborhoods = Dict()
inducing = []
for i in vertices(G)
    nb = neighbors(G, i)
    c = length(nb)
    if c >= threshold
        push!(inducing, i)
    end
    neighborhoods[i] = nb
end
#create a dictionary of the neighborhoods of the graph; a better implementation is possible but this works OK
hideg_graph = induced_subgraph(G, inducing)
#create an induced subgraph for the high degree nodes

function sparsetrianglefind(G)
    for i in edges(G)
        a = neighborhoods[i.first]
        b = neighborhoods[i.second]
        c = length(a)
        d = length(b)
        if c > threshold && d > threshold
    
        else 
            if c >= d
                for j in b
                    if in(j,a)
                        return i.first, i.second, j
                    end
                end
            else
                for j in a
                    if in(j,b)
                        return i.first, i.second, j
                    end
                end
            end
        end
    end
    return 0
end
#over the sparse graph components, test every edge's endpoints' neighborhood and look for a node in common
#we skip the dense edges for this

function densetrianglefind(hideg_graph)
    A = adjacency_matrix(hideg_graph)
    B = A*A
    for i = 1:size(A,1)
        for j = 1:size(A,1)
            if A[i,j] > 0 && B[i,j] > 0
                a = neighborhoods[i]
                b = neighborhoods[j]
                for k in b
                    if in(k,a)
                        return inducing[i], inducing[j], inducing[k]
                    end
                end
            end
        end
    end
    return "No Triangles"
end
#for the dense part, look at the graph induced over the dense nodes and do a search by matrix multiplication
#we know that i and j form a triangle with another vertex k iff A_i,j is nonzero and A^2_i,j is nonzero,
#where A is the adjacency matrix. Once such i and j are found, we check their neighborhoods for an intersection.

function trianglefind(G, hideg_graph)
    a = sparsetrianglefind(G)
    if a == 0
        return densetrianglefind(hideg_graph)
    else
        return a        
    end
end
#we first check the sparse componment; if no edge is found, check the dense component.

tic()
a = trianglefind(G, hideg_graph)
t = toq()
#test on our hybrid graph

print(a)


(156,709,979)

In [5]:
print("Time Elapsed: ")
println(t)
#even on graphs of a quarter million nodes, this algorithm runs in under a fifth of a second

Time Elapsed: 0.190297998
